In [7]:
from glob import glob
import string
import numpy as np
import tensorflow as tf
from datetime import datetime
from scipy.special import expit as sigmoid

In [2]:
!pip install tensorflow

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\mazic\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\~cipy\\.libs\\libansari.R6EA3HQP5KZ6TAXU4Y4ZVTRPT7UVA53Z.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\mazic\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Using legacy setup.py install for termcolor, since package 'wheel' is not installed.
Using legacy setup.py install for wrapt, since package 'wheel' is not installed.
    Running setup.py install for termcolor: started
    Running setup.py install for termcolor: finished with status 'done'
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2


In [9]:
def remove_punctuation(s):
  return s.translate(str.maketrans('','',string.punctuation))

In [10]:
def get_wiki():
  V = 20000
  files = glob('large_files/enwiki*.txt')
  all_word_counts = {}
  for f in files:
    for line in open(f, encoding="utf8"):
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          for word in s:
            if word not in all_word_counts:
              all_word_counts[word] = 0
            all_word_counts[word] += 1
  V = min(V, len(all_word_counts))
  all_word_counts = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)
  top_words = [w for w, count in all_word_counts[:V-1]] + ['<UNK>']
  word2idx = {w: i for i, w in enumerate(top_words)}
  unk = word2idx['<UNK>']
  
  sents = []
  for f in files:
    for line in open(f, encoding="utf8"):
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          sent = [word2idx[w] if w in word2idx else unk for w in s]
          sents.append(sent)
  return sents, word2idx
          
              

In [43]:
def get_wiki_small():
  V = 20
  file = 'large_files/enwiki-20180401-pages-articles1.xml-p10p30302-01.txt'
  all_word_counts = {}
  i = 0
  for line in open(file, encoding="utf8"):
    if (i < 10): 
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          i += 1
          for word in s:
            if word not in all_word_counts:
              all_word_counts[word] = 0
            all_word_counts[word] += 1
            
  V = min(V, len(all_word_counts))
  all_word_counts = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)
  top_words = [w for w, count in all_word_counts[:V-1]] + ['<UNK>']
  word2idx = {w: i for i, w in enumerate(top_words)}
  unk = word2idx['<UNK>']
  
  sents = []
  i = 0
  for line in open(file, encoding="utf8"):
    if (i < 10): 
      if line and line[0] not in '[*-|=\{\}':
        s = remove_punctuation(line).lower().split()
        if len(s) > 1:
          i += 1
          sent = [word2idx[w] if w in word2idx else unk for w in s]
          sents.append(sent)
  return sents, word2idx
          

In [19]:
def get_negative_sampling_distribution(sentences):
  # Pn(w) = prob of word occuring
  # we would like to sample the negative samples
  # such that words that occur more often
  # should be sampled more often

  word_freq = {}
  word_count = sum(len(sentence) for sentence in sentences)
  for sentence in sentences:
      for word in sentence:
          if word not in word_freq:
              word_freq[word] = 0
          word_freq[word] += 1
  
  # vocab size
  V = len(word_freq)

  p_neg = np.zeros(V)
  for j in range(V):
      p_neg[j] = word_freq[j]**0.75

  # normalize it
  p_neg = p_neg / p_neg.sum()

  assert(np.all(p_neg > 0))
  return p_neg

In [12]:
def get_context(pos, sentence, window_size):
  start = max(0, pos - window_size)
  end_  = min(len(sentence), pos + window_size)

  context = []
  for ctx_pos, ctx_word_idx in enumerate(sentence[start:end_], start=start):
    if ctx_pos != pos:
      context.append(ctx_word_idx)
  return context

In [38]:
def train_model():
  sentences, word2idx = get_wiki_small()
  tf.compat.v1.disable_eager_execution()
  # number of unique words
  vocab_size = len(word2idx)


  # config
  window_size = 5
  learning_rate = 0.025
  final_learning_rate = 0.0001
  num_negatives = 5 # number of negative samples to draw per input word
  samples_per_epoch = int(1e5)
  epochs = 20
  D = 5 # word embedding size

  # learning rate decay
  learning_rate_delta = (learning_rate - final_learning_rate) / epochs

  # distribution for drawing negative samples
  p_neg = get_negative_sampling_distribution(sentences)


  # params
  W = np.random.randn(vocab_size, D).astype(np.float32) # input-to-hidden
  V = np.random.randn(D, vocab_size).astype(np.float32) # hidden-to-output


  # create the model
  tf_input = tf.compat.v1.placeholder(tf.int32, shape=(None,))
  tf_negword = tf.compat.v1.placeholder(tf.int32, shape=(None,))
  tf_context = tf.compat.v1.placeholder(tf.int32, shape=(None,)) # targets (context)
  tfW = tf.Variable(W)
  tfV = tf.Variable(V.T)
  # biases = tf.Variable(np.zeros(vocab_size, dtype=np.float32))

  def dot(A, B):
    C = A * B
    return tf.reduce_sum(input_tensor=C, axis=1)

  # correct middle word output
  emb_input = tf.nn.embedding_lookup(params=tfW, ids=tf_input) # 1 x D
  emb_output = tf.nn.embedding_lookup(params=tfV, ids=tf_context) # N x D
  correct_output = dot(emb_input, emb_output) # N
  # emb_input = tf.transpose(emb_input, (1, 0))
  # correct_output = tf.matmul(emb_output, emb_input)
  pos_loss = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.ones(tf.shape(input=correct_output)), logits=correct_output)

  # incorrect middle word output
  emb_input = tf.nn.embedding_lookup(params=tfW, ids=tf_negword)
  incorrect_output = dot(emb_input, emb_output)
  # emb_input = tf.transpose(emb_input, (1, 0))
  # incorrect_output = tf.matmul(emb_output, emb_input)
  neg_loss = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.zeros(tf.shape(input=incorrect_output)), logits=incorrect_output)

  # total loss
  loss = tf.reduce_mean(input_tensor=pos_loss) + tf.reduce_mean(input_tensor=neg_loss)

  # output = hidden.dot(tfV)

  # loss
  # neither of the built-in TF functions work well
  # per_sample_loss = tf.nn.nce_loss(
  # # per_sample_loss = tf.nn.sampled_softmax_loss(
  #   weights=tfV,
  #   biases=biases,
  #   labels=tfY,
  #   inputs=hidden,
  #   num_sampled=num_negatives,
  #   num_classes=vocab_size,
  # )
  # loss = tf.reduce_mean(per_sample_loss)

  # optimizer
  # train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  train_op = tf.compat.v1.train.MomentumOptimizer(0.1, momentum=0.9).minimize(loss)
  # train_op = tf.train.AdamOptimizer(1e-2).minimize(loss)

  # make session
  session = tf.compat.v1.Session()
  init_op = tf.compat.v1.global_variables_initializer()
  session.run(init_op)


  # save the costs to plot them per iteration
  costs = []


  # number of total words in corpus
  total_words = sum(len(sentence) for sentence in sentences)
  print("total number of words in corpus:", total_words)


  # for subsampling each sentence
  threshold = 1e-5
  p_drop = 1 - np.sqrt(threshold / p_neg)


  # train the model
  for epoch in range(epochs):
    # randomly order sentences so we don't always see
    # sentences in the same order
    np.random.shuffle(sentences)

    # accumulate the cost
    cost = 0
    counter = 0
    inputs = []
    targets = []
    negwords = []
    t0 = datetime.now()
    for sentence in sentences:

      # keep only certain words based on p_neg
      sentence = [w for w in sentence \
        if np.random.random() < (1 - p_drop[w])
      ]
      if len(sentence) < 2:
        continue


      # randomly order words so we don't always see
      # samples in the same order
      randomly_ordered_positions = np.random.choice(
        len(sentence),
        # size=np.random.randint(1, len(sentence) + 1),
        size=len(sentence),
        replace=False,
      )


      for j, pos in enumerate(randomly_ordered_positions):
        # the middle word
        word = sentence[pos]

        # get the positive context words/negative samples
        context_words = get_context(pos, sentence, window_size)
        neg_word = np.random.choice(vocab_size, p=p_neg)

        
        n = len(context_words)
        inputs += [word]*n
        negwords += [neg_word]*n
        # targets = np.concatenate([targets, targets_])
        targets += context_words

        # _, c = session.run(
        #   (train_op, loss),
        #   feed_dict={
        #     tf_input: [word],
        #     tf_negword: [neg_word],
        #     tf_context: targets_,
        #   }
        # )
        # cost += c


      if len(inputs) >= 128:
        _, c = session.run(
          (train_op, loss),
          feed_dict={
            tf_input: inputs,
            tf_negword: negwords,
            tf_context: targets,
          }
        )
        cost += c

        # reset
        inputs = []
        targets = []
        negwords = []

      counter += 1
      if counter % 100 == 0:
        print("processed %s / %s\r" % (counter, len(sentences)))


    # print stuff so we don't stare at a blank screen
    dt = datetime.now() - t0
    print("epoch complete:", epoch, "cost:", cost, "dt:", dt)

    # save the cost
    costs.append(cost)

    # update the learning rate
    learning_rate -= learning_rate_delta


  # get the params
  W, VT = session.run((tfW, tfV))
  V = VT.T

  # return the model
  return word2idx, W, V

In [32]:
sentences, word2idx = get_wiki_small()
print(sentences, word2idx)

[[9, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], [3, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 3, 9, 0, 9, 7, 9, 9, 9, 4, 9], [9, 9, 7, 0, 9, 9, 9, 3, 9, 9, 9, 9, 9, 9, 9, 2, 0, 9, 1, 9, 9, 9, 3, 9, 9, 9, 8, 9, 9, 4, 9, 1, 6, 9, 4, 6, 9, 9, 9, 9, 9, 1, 9, 9, 9, 9, 9, 9, 9], [3, 9, 9, 9, 8, 9, 9, 1, 9, 9, 8, 9, 9, 9, 9, 9, 9, 4, 9, 5, 8, 9, 9, 9, 4, 9, 1, 3, 9, 9, 9, 1, 9, 9, 9, 9, 6, 9, 1, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 9, 9, 9, 1, 3, 9, 9, 9, 9, 9, 0, 9, 1, 9, 4, 9, 3, 9, 9, 9, 9], [0, 9, 3, 9, 9, 9, 0, 9, 9, 4, 0, 9, 9, 9, 9, 9, 9, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 0, 9, 9, 9, 9, 9, 9, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 4, 0, 9, 9, 9, 9, 9, 9, 9, 0, 9, 9, 9, 9, 9, 0, 9, 9, 9, 1, 9, 9, 9, 2, 9, 9, 9, 9, 0, 9, 9, 9, 9, 5, 9, 5, 9, 9, 9, 0, 9, 9, 9, 9, 9, 9, 1, 9, 9, 9, 9, 9, 9, 9, 1, 0, 9, 9, 9, 9, 9, 7, 0, 6, 9, 1, 0, 9, 9, 9, 5, 9, 9, 4, 9, 9, 9, 9, 9, 9, 9, 0, 9, 6, 9, 3, 2, 9, 9, 9, 9, 9], [0, 9, 9, 9, 7, 9

In [47]:
np.random.seed(0)
word2idx, W, V = train_model()
print(W)

total number of words in corpus: 784
epoch complete: 0 cost: 0 dt: 0:00:00.000997
epoch complete: 1 cost: 0 dt: 0:00:00.000997
epoch complete: 2 cost: 0 dt: 0:00:00
epoch complete: 3 cost: 0 dt: 0:00:00.000997
epoch complete: 4 cost: 0 dt: 0:00:00.000998
epoch complete: 5 cost: 0 dt: 0:00:00.000997
epoch complete: 6 cost: 0 dt: 0:00:00.000997
epoch complete: 7 cost: 0 dt: 0:00:00.000998
epoch complete: 8 cost: 0 dt: 0:00:00.001995
epoch complete: 9 cost: 0 dt: 0:00:00.000998
epoch complete: 10 cost: 0 dt: 0:00:00
epoch complete: 11 cost: 0 dt: 0:00:00.000998
epoch complete: 12 cost: 0 dt: 0:00:00
epoch complete: 13 cost: 0 dt: 0:00:00.000997
epoch complete: 14 cost: 0 dt: 0:00:00.000998
epoch complete: 15 cost: 0 dt: 0:00:00.000997
epoch complete: 16 cost: 0 dt: 0:00:00
epoch complete: 17 cost: 0 dt: 0:00:00.000997
epoch complete: 18 cost: 0 dt: 0:00:00
epoch complete: 19 cost: 0 dt: 0:00:00.000986
[[ 1.7640524   0.4001572   0.978738    2.2408931   1.867558  ]
 [-0.9772779   0.95008844

In [44]:
W = np.random.randn(6, 4)
V = np.random.randn(4, 6)
prob = sigmoid(W[3].dot(V[:,[2, 4]]))
print(prob)

gV = np.outer(W[3], prob - 1)
print(gV)

[0.79979108 0.33298967]
[[0.16974153 0.56550604]
 [0.15930054 0.53072116]
 [0.00803347 0.02676408]
 [0.07554102 0.2516703 ]]
